# Connect to SAP HANA Cloud

This notebook helps you connect to your SAP HANA Cloud instance and set up tables for vector embeddings.

In [ ]:
# Import required libraries
import os
import sys
import logging
from dotenv import load_dotenv

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger("hana-connection")

## Set Your HANA Cloud Credentials

Enter your SAP HANA Cloud credentials below:

In [ ]:
# Enter your HANA Cloud credentials
os.environ["HANA_HOST"] = "your-hana-hostname.hanacloud.ondemand.com"  # Replace with your HANA host
os.environ["HANA_PORT"] = "443"                                        # Usually 443 for HANA Cloud
os.environ["HANA_USER"] = "your-username"                              # Replace with your username
os.environ["HANA_PASSWORD"] = "your-password"                          # Replace with your password
os.environ["DEFAULT_TABLE_NAME"] = "EMBEDDINGS"                        # Table name for vector storage

# Save to .env file for future use
with open(".env", "w") as f:
    f.write(f"HANA_HOST={os.environ['HANA_HOST']}\n")
    f.write(f"HANA_PORT={os.environ['HANA_PORT']}\n")
    f.write(f"HANA_USER={os.environ['HANA_USER']}\n")
    f.write(f"HANA_PASSWORD={os.environ['HANA_PASSWORD']}\n")
    f.write(f"DEFAULT_TABLE_NAME={os.environ['DEFAULT_TABLE_NAME']}\n")
    f.write("TEST_MODE=false\n")
    f.write("ENABLE_CORS=true\n")
    f.write("LOG_LEVEL=INFO\n")

print("Credentials saved to .env file")

## Test Connection to HANA Cloud

Now let's test the connection to your HANA Cloud instance:

In [ ]:
# Install required packages if not already installed
!pip install hdbcli python-dotenv

In [ ]:
from hdbcli import dbapi

def test_hana_connection():
    """Test connection to SAP HANA Cloud"""
    host = os.environ["HANA_HOST"]
    port = int(os.environ["HANA_PORT"])
    user = os.environ["HANA_USER"]
    password = os.environ["HANA_PASSWORD"]
    
    logger.info(f"Testing connection to SAP HANA Cloud at {host}:{port}")
    try:
        # Establish connection
        connection = dbapi.connect(
            address=host,
            port=port,
            user=user,
            password=password,
            encrypt=True,
            sslValidateCertificate=True
        )
        
        # Execute a simple query to verify connection
        cursor = connection.cursor()
        cursor.execute('SELECT VERSION FROM SYS.M_DATABASE')
        version = cursor.fetchone()[0]
        
        # Get schema information
        cursor.execute("SELECT SCHEMA_NAME FROM SYS.SCHEMAS WHERE SCHEMA_OWNER = CURRENT_USER")
        schemas = cursor.fetchall()
        schema_list = [row[0] for row in schemas]
        
        # Get table information from user schemas
        tables = []
        for schema in schema_list:
            cursor.execute(f"SELECT TABLE_NAME FROM SYS.TABLES WHERE SCHEMA_NAME = '{schema}' AND IS_USER_DEFINED_TYPE = 'FALSE'")
            schema_tables = cursor.fetchall()
            for table in schema_tables:
                tables.append(f"{schema}.{table[0]}")
        
        cursor.close()
        connection.close()
        
        logger.info(f"Successfully connected to SAP HANA Cloud. Version: {version}")
        logger.info(f"User schemas: {', '.join(schema_list)}")
        logger.info(f"Found {len(tables)} tables in user schemas")
        if tables:
            logger.info(f"Tables: {', '.join(tables[:10])}" + (f" and {len(tables)-10} more..." if len(tables) > 10 else ""))
        
        return True, connection, version, schema_list, tables
    
    except Exception as e:
        logger.error(f"Failed to connect to SAP HANA Cloud: {str(e)}")
        return False, None, None, None, None

# Test the connection
success, conn, version, schemas, tables = test_hana_connection()

if success:
    print("✅ Connection successful!")
    print(f"HANA version: {version}")
    print(f"Available schemas: {schemas}")
else:
    print("❌ Connection failed. Please check your credentials.")

## Create Table for Vector Embeddings

Now let's create a table for storing vector embeddings:

In [ ]:
def create_embeddings_table(schema_name, table_name, embedding_size=384):
    """Create a table for storing embeddings"""
    host = os.environ["HANA_HOST"]
    port = int(os.environ["HANA_PORT"])
    user = os.environ["HANA_USER"]
    password = os.environ["HANA_PASSWORD"]
    
    try:
        # Connect to HANA
        connection = dbapi.connect(
            address=host,
            port=port,
            user=user,
            password=password,
            encrypt=True,
            sslValidateCertificate=True
        )
        
        cursor = connection.cursor()
        
        # Create schema if it doesn't exist
        try:
            cursor.execute(f"CREATE SCHEMA {schema_name}")
            logger.info(f"Created schema {schema_name}")
        except Exception as e:
            if "exists" in str(e).lower():
                logger.info(f"Schema {schema_name} already exists")
            else:
                logger.error(f"Error creating schema {schema_name}: {str(e)}")
                raise
        
        # Create embeddings table with the HANA vector engine
        full_table_name = f"{schema_name}.{table_name}"
        try:
            # Drop table if it exists
            cursor.execute(f"DROP TABLE {full_table_name}")
            logger.info(f"Dropped existing table {full_table_name}")
        except Exception as e:
            if "not found" not in str(e).lower():
                logger.warning(f"Note: {str(e)}")
        
        # Create table with document ID, content, metadata, and embedding vector
        create_table_sql = f"""
        CREATE TABLE {full_table_name} (
            ID VARCHAR(100) PRIMARY KEY,
            DOCUMENT_CONTENT NCLOB,
            METADATA NCLOB,
            EMBEDDING REAL ARRAY({embedding_size})
        )
        """
        cursor.execute(create_table_sql)
        logger.info(f"Created table {full_table_name}")
        
        # Create vector index for similarity search
        try:
            create_index_sql = f"""
            CREATE HNSW INDEX IDX_EMBEDDING_HNSW ON {full_table_name}(EMBEDDING)
            PARAMETERS ('M'='16', 'ef_construction'='64', 'ef'='40', 'distance_measure'='cosine')
            """
            cursor.execute(create_index_sql)
            logger.info(f"Created HNSW vector index on {full_table_name}")
        except Exception as e:
            logger.warning(f"Could not create HNSW index: {str(e)}")
            logger.warning("Your HANA instance might not support vector search or HNSW indexes.")
            
        connection.commit()
        cursor.close()
        connection.close()
        
        return True
    except Exception as e:
        logger.error(f"Error creating embeddings table: {str(e)}")
        return False

# If you're connected successfully, you can create the embeddings table
if success and schemas:
    # Use the first available schema or enter your preferred schema
    schema_name = schemas[0]  # Or replace with your preferred schema
    table_name = os.environ["DEFAULT_TABLE_NAME"]
    embedding_size = 384  # Default size for most embedding models
    
    print(f"Creating table {schema_name}.{table_name} for embeddings...")
    table_created = create_embeddings_table(schema_name, table_name, embedding_size)
    
    if table_created:
        print(f"✅ Table {schema_name}.{table_name} created successfully!")
    else:
        print("❌ Failed to create table.")

## Using the Table with LangChain

Once you have your table set up, you can use it with LangChain for vector storage:

In [ ]:
# Install required packages
!pip install langchain sentence-transformers

In [ ]:
# Import the HanaVectorStore module
from langchain_hana.vectorstores import HanaVectorStore
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize embeddings model
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Initialize HANA vector store
vector_store = HanaVectorStore(
    host=os.environ["HANA_HOST"],
    port=int(os.environ["HANA_PORT"]),
    user=os.environ["HANA_USER"],
    password=os.environ["HANA_PASSWORD"],
    schema_name=schema_name,  # Use the schema name from earlier
    table_name=table_name,    # Use the table name from earlier
    embedding=embeddings
)

# Add documents
docs = [
    "SAP HANA is a high-performance in-memory database",
    "LangChain provides a framework for LLM applications",
    "Vector databases are optimized for similarity search"
]

vector_store.add_texts(docs)
print(f"✅ Added {len(docs)} documents to the vector store")

# Perform similarity search
query = "What is SAP HANA?"
results = vector_store.similarity_search(query, k=1)
print(f"\nQuery: {query}")
print(f"Result: {results[0].page_content}")